In [64]:
import requests
import json
import os
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

True

In [65]:
base_url = "https://jira.eficode.com"
access_token = os.getenv('JIRA_API_TOKEN')


In [66]:
url = f'{base_url}/rest/tempo-planning/1/allocation'
header = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}
res = requests.get(url=url, headers=header)

print(res.status_code)
data = res.json()

200


In [67]:
df = pd.DataFrame(columns=['plan_id', 'employee', 'start', 'end', 'hours_day', 'employee_day'])


for plan in data:
    row = {'plan_id': plan['id'], 'employee': plan['assignee']['key'], 'start': plan['start'], 'end': plan['end'], 'hours_day': int(plan['secondsPerDay'])/(60*60)}
    df.loc[len(df)] = row

df['start'] = pd.to_datetime(df['start'], format='%Y/%m/%d')
df['end'] = pd.to_datetime(df['end'], format='%Y/%m/%d')

# Drop rows where the value in the 'employee' column equals a certain value
value_to_drop = '1'
df = df[df['employee'] != value_to_drop]
# dfgroup = df

# df

In [68]:




employee_data = df.copy()
# employee_data



In [69]:
''

df['date_range'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('date_range')

df_summed = df_exploded.groupby(['date_range', 'employee'])['hours_day'].sum().reset_index()
# df_summed
plans = df_exploded.groupby(['date_range', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()
# Merge the new data frame with the employee data
merged_data = pd.merge(df_summed, plans, how='left')
# Read in the date table
date_table = pd.read_csv('Datetable.csv', parse_dates=['Date'])
# Convert the date column to a datetime object
date_table['Date'] = pd.to_datetime(date_table['Date'], format='%Y/%m/%d')

date_table.set_index('Date', inplace=True)


df['date'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('date')

df_hours_sum = df_exploded.groupby(['date', 'employee'])['hours_day'].sum().reset_index()
df_plan_agg = df_exploded.groupby(['date', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()

df_merged = pd.merge(df_hours_sum, df_plan_agg, on=['date', 'employee'])
# Read in the date table and employee data table
date_df = pd.read_csv('Datetable.csv')

begin = '1/1/2022'
end = '31/12/2024'

all_dates = pd.date_range(start=begin, end=end, freq='D')
all_names = df_merged['employee'].unique()
name_date_df = pd.MultiIndex.from_product([all_names, all_dates], names=['employee', 'date']).to_frame(index=False)

df_all_days = pd.merge(name_date_df, df_merged, on=['employee', 'date'], how='left').fillna({'hours_day': 0})

df_all_days['date'] = df_all_days['date'].dt.floor('D')
df_all_days[df_all_days['employee'] == 'tuomas.kara']


# row.info()
# dates_df.info()
merged_data = df_all_days.copy()

merged_data.set_index('date', inplace=True)

merged_data['allocation'] = 'Not fully allocated'

merged_data.loc[merged_data['hours_day'] >= 7.5, 'allocation'] = 'Fully allocated'
merged_data.loc[merged_data['hours_day'] <= 0.1, 'allocation'] = 'Not allocated'

merged_data['is_workday'] = False
# merged_data.loc[date_table['is_workday'] == True, 'is_workday'] = True
# merged_data.loc[date_table['is_workday'] == False, 'is_workday'] = False
""" merged_data['coefficient'] = 0.5



merged_data.loc[merged_data['hours_day'] >= 7.5, 'coefficient'] = 1.0
merged_data.loc[merged_data['hours_day'] <= 0.1, 'coefficient'] = 0.0 """

# df_all_days = pd.merge(merged_data, date_table, on=['is_workday'], how='left')

# merged_data.loc[merged_data['hours_day'] < 7.5 and merged_data['hours_day'] > 0.1 , 'allocation'] = 'Partly allocated'
# merged_data['allocation'] = ''

# date_table = pd.read_csv('Datetable.csv', parse_dates=['Date'])

# rowvalue
# plans
# df_summed
# merged_data

# Write the updated data frame to a new CSV file
# merged_data.to_csv('updated_employee_data.csv', index=False)'Api.ipynb
# selecting old value
# a = merged_data['is_workday']

# selecting new value
# b = date_table['is_workday']


# replace values of one DataFrame
# with the value of another DataFrame
# merged_data = merged_data.replace(a,b)

merged_data['is_workday'] =date_table['is_workday']


# Display the Output
# display(merged_data)


merged_data.to_csv('updated_employee_data.csv', index=True)


C:\Users\JariKaijansinkko\AppData\Local\Temp\ipykernel_14396\4107993517.py:34: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  all_dates = pd.date_range(start=begin, end=end, freq='D')


In [70]:
# merged_data.groupby('employee').sum()

# pivot_data

# date_range

In [71]:
res = requests.get(f'{base_url}/rest/tempo-teams/2/team', headers=header)
teams = res.json()

In [72]:
team_members = requests.get(f'{base_url}/rest/tempo-teams/2/team/42/member', headers=header).json()
for member in team_members:
    print(member['member']['name'])

jani.lundan
jarkko.sillanpaa
juha.patrikainen
juho.lehtonen
jukka.haavisto
jyri.tienhaara
kai.jokiniemi
matias.ijas
mika.tavi
tapio.tuomisto
teemu.partanen
tuomas.kara
vilma.pohjonen
